In [277]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import re
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [278]:
df=pd.read_csv("sphist.csv")
df['Date']=pd.to_datetime(df['Date'])
df.sort_values(by='Date',ascending=True,inplace=True)
df=df.reset_index(drop=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [279]:
df['Date'].head()

0   1950-01-03
1   1950-01-04
2   1950-01-05
3   1950-01-06
4   1950-01-09
Name: Date, dtype: datetime64[ns]

In [280]:
def prev_year(date):
    prevyear=pd.Timestamp(date.year-1,date.month,date.day)
    return prevyear
    

In [281]:
df['yr_avg']=1
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,yr_avg
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,1
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,1
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,1
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,1
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,1


In [302]:
for index,row in df.iterrows():
    if index<250:
        df.loc[index,'yr_avg']=0
        df.loc[index,'five_day']=0
        df.loc[index,'std']=0
    else:
        last_five_days=df.loc[index-6:index]
        five_day=last_five_days['Close'].mean()
        five_day_std=last_five_days['Close'].std()
        five_day_volume=last_five_days['Volume'].mean()
        df.loc[index,'vol_avg']=five_day_volume
        df.loc[index,'std']=five_day_std
        df.loc[index,'five_day']=five_day
        date=df.loc[index,'Date']
        if date.day!=29:
            prevyear=prev_year(date)
            prev_df=df[(df['Date']>=prevyear) & (df['Date']<date)]
            yr_avg=prev_df['Close'].mean()
            df.loc[index,'yr_avg']=yr_avg
        if date.day==29:
            new_date=date-timedelta(days=1)
            prevyear=prev_year(new_date)
            prev_df=df[(df['Date']>=prevyear) & (df['Date']<date)]
            yr_avg=prev_df['Close'].mean()
            df.loc[index,'yr_avg']=yr_avg
            
            

In [235]:
df['five_day']=np.nan

In [268]:
for index,row in df.iterrows():
    if index<250:
        df.loc[index,'five_day']=0
    else:
        last_five_days=df.loc[index-6:index]
        five_day=last_five_days['Close'].mean()


In [291]:
for index,row in df.iterrows():
    df.loc[index,'day_of_week']=df.loc[index,'Date'].dayofweek

In [303]:
data=df[df["Date"] > datetime(year=1951, month=1, day=2)]


In [305]:
data=data.dropna(axis=0)
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close,yr_avg,five_day,day_of_week,std,vol_avg
250,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,18.406760,20.365714,2.0,0.306424,3.102857e+06
251,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,18.422880,20.480000,3.0,0.326293,3.198571e+06
252,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,18.438960,20.615714,4.0,0.240961,3.302857e+06
253,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,18.460643,20.715715,0.0,0.233228,3.280000e+06
254,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,18.470800,20.821429,1.0,0.223191,3.314286e+06


In [306]:
train=data[data['Date'] < datetime(year=2013, month=1 , day=1)]
test=data[data['Date'] >= datetime(year=2013, month=1 , day=1)]

In [310]:
features=['yr_avg','five_day','std']
lr=LinearRegression()
lr.fit(train[features],train['Close'])
predictions=lr.predict(test[features])
mas=mean_absolute_error(test['Close'],predictions)
mas

14.650947974078699